In [10]:
import glob
import os
import fuse, causal_utils
glob.glob("/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/*/")

['/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_3/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_2/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_1/']

In [11]:
!ls /raid/can/nli_models/poe3

nli  outputs_poe_bert_base_korn_clark_1_seed33370


In [12]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from scipy.special import expit

# select model
model_path='/raid/can/nli_models/poe3/nli/outputs_poe_bert_base_korn_clark_1_seed13370/'
# select data path
data_path='/ist/users/canu/debias_nlu/data/nli/'
df = pd.read_json(model_path+'raw_train.jsonl', lines=True)
# select fusion method here
fusion = fuse.sum_fuse

#  get score from the bias model

In [13]:
# avg prob
#s score from bias model
df_hans = pd.read_json(data_path+'hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)
hans_score=[b for b in df_hans['bias_probs'] ]
hans_score=np.array(hans_score)

# Get avg  prob of bert

In [14]:
list_probs = []
for i in df['probs']:
    list_probs.extend(i)
x=np.array(list_probs)
avg=np.average(x,axis=0)

# y0m0

In [15]:
bias_score=fusion(avg,hans_score)
#y0m0

In [16]:
!ls /raid/can/nli_models/poe3/outputs_poe_bert_base_korn_clark_1_seed13370/

ls: cannot access '/raid/can/nli_models/poe3/outputs_poe_bert_base_korn_clark_1_seed13370/': No such file or directory


In [17]:
result_path=model_path+'normal/'
df_bert = pd.read_json(result_path+'hans_result.jsonl', lines=True)

In [18]:
# ent = []
y1m1prob = []
for p,h in zip(df_bert['probs'],hans_score):
    new_y1m1 = fusion(np.array(p),h)
    y1m1prob.append(new_y1m1)
  

# corrected y0m0

In [19]:
c = sharpness_correction(hans_score, y1m1prob) 
bias_score = fusion(c,hans_score)

NameError: name 'sharpness_correction' is not defined

# TIE

In [22]:
debias_scores = []
for p,b in zip(y1m1prob,bias_score):
    debias_scores.append(p-b) # TIE  

# Eval HANS

In [23]:
key = {0:"entailment",1:"contradiction",2:"neutral"}
labels = []
for i in debias_scores:
    labels.append(key[np.argmax(i)])
df_bert['debias_label']=labels

In [24]:
text_ans=""
for idx, obj in enumerate(df_bert['label']):
    text_ans = text_ans + "ex"+str(idx)+","+obj+"\n"   
    
text_ans_debias=""
for idx, obj in enumerate(df_bert['debias_label']):
    text_ans_debias = text_ans_debias + "ex"+str(idx)+","+obj+"\n"          

In [25]:
def format_label(label):
    if label == "entailment":
        return "entailment"
    else:
        return "non-entailment"

guess_dict = {}
for line in text_ans.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict[parts[0]] = format_label(parts[1])
        
guess_dict_debias = {}
for line in text_ans_debias.split("\n"):
    if len(line)>1:
        parts = line.strip().split(",")
        guess_dict_debias[parts[0]] = format_label(parts[1])        

In [26]:
labels,baseline_avg=causal_utils.get_heur(guess_dict) # normal

Heuristic entailed results:
lexical_overlap: 0.9734
subsequence: 0.9968
constituent: 0.9924

Heuristic non-entailed results:
lexical_overlap: 0.7482
subsequence: 0.1114
constituent: 0.3026
avg: 0.6874666666666668


In [27]:
labels,debias_avg=causal_utils.get_heur(guess_dict_debias) # TIE

Heuristic entailed results:
lexical_overlap: 0.9724
subsequence: 0.9966
constituent: 0.9912

Heuristic non-entailed results:
lexical_overlap: 0.7528
subsequence: 0.116
constituent: 0.3132
avg: 0.6903666666666667


# Causal Mediation Analysis

In [28]:
def get_ans(ans):
    if ans == 0:
        return 'entailment'
    else:
        return 'non-entailment' 
    
# y0m0
import pickle
modelname='mnli_lr_model.sav'
loaded_model = pickle.load(open(modelname, 'rb'))
x0=loaded_model.predict_proba(np.array([[0,0,0.41997876976119086]]))
m0=avg
y0m0=fusion(x0,m0)    

In [29]:
factual_pred_correct = []
TIE_pred_correct = []
NIE_pred_correct = []
INTmed_pred_correct = []
pred_correct = []
all_TIE = []
all_NIE = []
all_NDE = []
all_INTmed = []
for i in range(len(labels)): 
    y1m1 = y1m1prob[i]
    y1m0 = bias_score[i]
    TE = y1m1 - y0m0
    NDE = bias_score[i] - y0m0
    y0m1= fusion(x0,np.array(df_bert['probs'][i]) )
    TIE = y1m1 - y1m0
    NIE = y0m1 - y0m0
    INTmed = TIE - NIE
    # factual
    factual_ans = np.argmax(df_bert['probs'][i])
    factual_ans = get_ans(factual_ans)
    factual_correct = factual_ans==labels[i]   
    factual_pred_correct.append(factual_correct)
    # TIE
    TIE_ans = np.argmax(TIE)
    TIE_ans = get_ans(TIE_ans)
    TIE_correct = TIE_ans==labels[i]  
    TIE_pred_correct.append(TIE_correct)
    # INTmed
    INTmed_ans = np.argmax(INTmed[0])
    INTmed_ans = get_ans(INTmed_ans)
    INTmed_correct = INTmed_ans==labels[i]  
    INTmed_pred_correct.append(INTmed_correct)
    # NIE
    NIE_ans = np.argmax(NIE[0])
    NIE_ans = get_ans(NIE_ans)
    NIE_correct = NIE_ans==labels[i]  
    NIE_pred_correct.append(NIE_correct)    
    
    # save
    all_NDE.append(NDE[0][0])
    all_NIE.append(NIE[0][0])
    all_TIE.append(TIE[0])
    all_INTmed.append((INTmed[0][0]))
    if  (TIE[0]/TE[0][0])<9999999:
        cf_ans = np.argmax(np.array(df_bert['probs'][i]-hans_score[i]))
        cf_ans = get_ans(cf_ans)  
        cf_correct = cf_ans==labels[i]
    else:
#         print(cf_ans)
        cf_correct = factual_ans ==labels[i]
    pred_correct.append(cf_correct)
    
#     np.array(df_bert['probs'][i]-bias_score)
#     labels[i]
print(np.array(all_TIE).mean(),(np.array(all_TIE).std()))
print(np.array(all_INTmed).mean(),(np.array(all_INTmed).std()))
print(np.array(all_NIE).mean(),(np.array(all_NIE).std()))

0.08649503037239671 0.07753374427816435
-0.03193751895445736 0.026914285032747287
0.11843254932685407 0.10443938146612336


In [30]:
 np.argmax(NIE)

0

In [31]:
NIE

array([[ 0.18147684, -0.12462973, -0.12555245]])

In [32]:
print(sum(NIE_pred_correct)/30000,sum(INTmed_pred_correct)/30000)

0.6872666666666667 0.3651


In [33]:
print(sum(factual_pred_correct)/30000,sum(pred_correct)/30000,sum(TIE_pred_correct)/30000)

0.6874666666666667 0.7312 0.6903666666666667


In [34]:
#TIE # 0.032416620996870094 # utama best seed  0.04260506151060745
#TIE # 0.0879242620437139 # PoE
#TIE # 0.10864416858494 # Baseline 0.10295758381662952
#
# baseline
# reweight_korn
# poe_korn


In [35]:
!python cma.py

Heuristic entailed results:
lexical_overlap: 0.9838
subsequence: 0.9946
constituent: 0.9978

Heuristic non-entailed results:
lexical_overlap: 0.7264
subsequence: 0.1144
constituent: 0.2728
avg: 0.6816333333333334
0.09049318935295611 0.07330273608231962
-0.033259518297814955 0.025551918800548676
0.12375270765077105 0.09884589944548612
Heuristic entailed results:
lexical_overlap: 0.9784
subsequence: 0.9986
constituent: 0.9998

Heuristic non-entailed results:
lexical_overlap: 0.6494
subsequence: 0.0844
constituent: 0.1872
avg: 0.6496333333333333
0.09638890603753264 0.06965581836013897
-0.035384280694031746 0.024282320036851667
0.13177318673156438 0.09392968074471238
Heuristic entailed results:
lexical_overlap: 0.9756
subsequence: 0.9966
constituent: 0.9952

Heuristic non-entailed results:
lexical_overlap: 0.7424
subsequence: 0.1658
constituent: 0.285
avg: 0.6934333333333332
0.0880008136703304 0.07883599964955158
-0.03250504798919367 0.027354145127177664
0.1205058616595241 0.10618183988554